In [ ]:
# Configuration 
import minio 
import docker 
minio_ip_address = "minio"

In [ ]:
from minio import Minio
client_minio = Minio(
    f"{minio_ip_address}:9000",
    access_key="root",
    secret_key="password",
    secure=False
)

# Création du bucket tp6
if client_minio.bucket_exists("tp6") == False:
    client_minio.make_bucket("tp6")

In [ ]:
from datetime import datetime
import time
import requests 
import json 
import io

# URL de l'API
API_URL = "https://open.tan.fr/ewp/tempsattentelieu.json/CTRE/20"

# Fonction pour interroger l'API et envoyer les données à MinIO
def fetch_and_upload_api_data():
    try:
        response = requests.get(API_URL)
        if response.status_code == 200:
            data = response.json()
            if data:
                # Préparer le fichier JSON à uploader
                timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H-%M-%S")
                file_key = f"naolib/bus_data_{timestamp}.json"
                json_data = "\n".join([json.dumps(record) for record in data])
                value_as_bytes = json_data.encode('utf-8')
                value_as_a_stream = io.BytesIO(value_as_bytes)

                # Upload des données dans MinIO
                client_minio.put_object(
                    bucket_name="tp6",
                    object_name=file_key,
                    data=value_as_a_stream,
                    length=len(value_as_bytes),
                    content_type="application/json"
                )

                print(f"Données ajoutées dans MinIO sous {file_key}.")
            else:
                print("Aucune donnée à envoyer.")
        else:
            print(f"Erreur HTTP : {response.status_code}")
    except Exception as e:
        print(f"Erreur lors de l'accès à l'API ou à MinIO : {e}")

# Interrogation périodique
while True:
    fetch_and_upload_api_data()
    time.sleep(10)  # Intervalle de 10 secondes